In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import xgboost as xgb 
from imblearn.combine import SMOTEENN

import plotly.graph_objects as go

In [3]:
path = '/Users/ruruthapa/Desktop/ML/machinelearning/Telecom_Churn/venv/Telecom_churn_final'
path_lat = '/Users/ruruthapa/Desktop/ML/machinelearning/Telecom_Churn/venv/archive-2/telecom_zipcode_population.csv'

In [4]:
df = pd.read_csv(path)

In [5]:
Drop_col = ['City','Offer','Zip Code','Longitude','Latitude','Churn Reason','Unnamed: 0']

In [6]:
df.drop(columns=Drop_col, inplace=True)

In [7]:
model_col = ['Customer ID','Gender', 'Age', 'Married', 'Number of Dependents',
        'Tenure in Months', 'Phone Service','Internet Service','Streaming Music'
       ,'Unlimited Data', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charge','Total Revenue', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies','Customer Status', 'Churn Category']

In [8]:
model_df = df[model_col]

for idx, val in enumerate(model_df['Number of Dependents']):
  if val==0:
    model_df['Number of Dependents'][idx]='No'
  else:
    model_df['Number of Dependents'][idx]= 'Yes'

/var/folders/1v/57gd2jmd04lg8mxrdxd44x300000gn/T/ipykernel_48258/921708357.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Number of Dependents'][idx]='No'
/var/folders/1v/57gd2jmd04lg8mxrdxd44x300000gn/T/ipykernel_48258/921708357.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Number of Dependents'][idx]= 'Yes'


In [9]:
model_df.head(4)

,Customer ID,Gender,Age,Married,Number of Dependents,Tenure in Months,Phone Service,Internet Service,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Revenue,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Customer Status,Churn Category
0,0002-ORFBO,Female,37,Yes,No,9,Yes,Yes,No,Yes,One Year,Yes,Credit Card,65.6,974.81,No,No,Yes,No,Yes,Yes,No,Stayed,NaN
1,0003-MKNFE,Male,46,No,No,9,Yes,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.0,610.28,Yes,No,No,No,No,No,Yes,Stayed,NaN
2,0004-TLHLJ,Male,50,No,No,4,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.9,415.45,No,No,No,Yes,No,No,No,Churned,Competitor
3,0011-IGKFF,Male,78,Yes,No,13,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1599.51,No,No,Yes,Yes,No,Yes,Yes,Churned,Dissatisfaction


In [10]:
category_mapping = {'0':0, 'Competitor': 1, 'Dissatisfaction': 2, 'Attitude': 3, 'Price': 4, 'Other': 5}
model_df['Churn Category'] = model_df['Churn Category'].fillna(0)
model_df['Churn Category']= model_df['Churn Category'].replace(category_mapping).astype(int)

class_mapping = {'Joined': 0, 'Stayed': 0, 'Churned': 1}
model_df['Customer Status'] = model_df['Customer Status'].map(class_mapping)

/var/folders/1v/57gd2jmd04lg8mxrdxd44x300000gn/T/ipykernel_48258/172829272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Churn Category'] = model_df['Churn Category'].fillna(0)
/var/folders/1v/57gd2jmd04lg8mxrdxd44x300000gn/T/ipykernel_48258/172829272.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Churn Category']= model_df['Churn Category'].replace(category_mapping).astype(int)
/var/folders/1v/57gd2jmd04lg8mxrdxd44x300000gn/T/ipykernel_48258/172829272.py:6: SettingWithCopyWar

In [11]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer ID           7043 non-null   object 
 1   Gender                7043 non-null   object 
 2   Age                   7043 non-null   int64  
 3   Married               7043 non-null   object 
 4   Number of Dependents  7043 non-null   object 
 5   Tenure in Months      7043 non-null   int64  
 6   Phone Service         7043 non-null   object 
 7   Internet Service      7043 non-null   object 
 8   Streaming Music       7043 non-null   object 
 9   Unlimited Data        7043 non-null   object 
 10  Contract              7043 non-null   object 
 11  Paperless Billing     7043 non-null   object 
 12  Payment Method        7043 non-null   object 
 13  Monthly Charge        7043 non-null   float64
 14  Total Revenue         7043 non-null   float64
 15  MultipleLines        

**Model Building**

In [12]:
X, y = model_df.drop(['Customer ID', 'Customer Status', 'Churn Category'], axis=1), model_df[['Customer Status','Churn Category']]

In [13]:
cat_cols = [col for col in X.columns if X[col].dtype == 'object']

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
trans_data = OH_encoder.fit_transform(X[cat_cols])

encode_col = OH_encoder.get_feature_names_out(input_features=cat_cols)
passthrough_col = [col for col in X.columns if col not in cat_cols]
new_cols = list(encode_col) + passthrough_col

data = np.hstack((trans_data, X[passthrough_col].values))

X_enc = pd.DataFrame(data,columns = new_cols)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
X_enc.shape

(7043, 40)

**Resampling w.r.t Target1**

In [15]:
sm = SMOTEENN(random_state = 42) 
# Sampling_strategy is left 'auto' which is equivalent to 'not majority', i.e., multiple copies of minority class will be generated to match the number of samples in majority class
# Then ENN  will be used to remove the misclassified majority samples
X_tgt1, y_tgt1 = sm.fit_resample(X_enc, y.iloc[:,0])

In [16]:
cols = X_enc.columns # keeping the names of all columns

scaler = MinMaxScaler()
Xtgt1_scaled = scaler.fit_transform(X_tgt1)
X_tgt1_scaled = pd.DataFrame(Xtgt1_scaled, columns=cols)

In [17]:
X_train1,X_test1, y_train1, y_test1 = train_test_split(X_tgt1_scaled, y_tgt1, test_size=0.2)

In [18]:
model_target1 = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=np.nan, monotone_constraints='()',
              n_estimators=165, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity = 0, eval_metric='logloss')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [19]:
model_target1.fit(X_train1, y_train1)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=0.2, gpu_id=-1,
              grow_policy=None, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=0,
              max_depth=8, max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=165, n_jobs=4,
              num_parallel_tree=1, predictor=None, random_state=0, ...)

In [20]:
feat_imp = model_target1.feature_importances_
feat_names = model_target1.feature_names_in_
feat_df = pd.DataFrame(feat_imp, index=feat_names, columns=['Importance'])
feat_df = feat_df.sort_values(by='Importance',ascending=False)
feat_df['Importance'] = feat_df['Importance'].multiply(100)
feat_df.index[:12]

Index(['Contract_Month-to-Month', 'Internet Service_Yes', 'Contract_Two Year',
       'Internet Service_No', 'Number of Dependents_Yes', 'Contract_One Year',
       'Tenure in Months', 'Payment Method_Credit Card',
       'Number of Dependents_No', 'StreamingMovies_Yes', 'OnlineSecurity_Yes',
       'Phone Service_No'],
      dtype='object')

In [21]:
accuracy_score(y_test1, model_target1.predict(X_test1))
print(classification_report(y_test1, model_target1.predict(X_test1)))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94       532
           1       0.95      0.97      0.96       679

    accuracy                           0.95      1211
   macro avg       0.95      0.95      0.95      1211
weighted avg       0.95      0.95      0.95      1211



In [23]:
model_target1.feature_importances_

array([0.00516577, 0.00491192, 0.00501593, 0.00627736, 0.01544258,
       0.05149314, 0.01262863, 0.00942653, 0.05317619, 0.11082929,
       0.00396102, 0.00779575, 0.00597428, 0.00611918, 0.41269156,
       0.02942336, 0.05888088, 0.00853427, 0.01102073, 0.00625402,
       0.01898052, 0.00678375, 0.01030727, 0.00437495, 0.01026585,
       0.01511068, 0.00583889, 0.00592821, 0.00314612, 0.0031156 ,
       0.00870371, 0.00606454, 0.00879188, 0.0047688 , 0.00932406,
       0.01535313, 0.00378662, 0.01930202, 0.0091373 , 0.00589368],
      dtype=float32)

**Resampling w.r.t Target2**

In [24]:
sm = SMOTEENN(random_state = 42) 
# Sampling_strategy is left 'auto' which is equivalent to 'not majority', i.e., multiple copies of minority class will be generated to match the number of samples in majority class
# Then ENN  will be used to remove the misclassified majority samples
X_tgt2, y_tgt2 = sm.fit_resample(X_enc, y.iloc[:,1])

cols2 = X_enc.columns # keeping the names of all columns

scaler = MinMaxScaler()
Xtgt2_scaled = scaler.fit_transform(X_tgt2)
X_tgt2_scaled = pd.DataFrame(Xtgt2_scaled, columns=cols2)

X_train2 ,X_test2, y_train2, y_test2 = train_test_split(X_tgt1_scaled, y_tgt1, test_size=0.2)

model_target2 = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=np.nan, monotone_constraints='()',
              n_estimators=165, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity = 0, eval_metric='logloss')

model_target2.fit(X_train2, y_train2)

accuracy_score(y_test2, model_target1.predict(X_test2))
print(classification_report(y_test2, model_target1.predict(X_test2)))


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


              precision    recall  f1-score   support

           0       0.99      0.98      0.98       532
           1       0.98      0.99      0.99       679

    accuracy                           0.99      1211
   macro avg       0.99      0.99      0.99      1211
weighted avg       0.99      0.99      0.99      1211



**Predicting Using Pipeline**

In [25]:
class CustomPipeline:
    
    def __init__(self, X ,y):
        
        self.col = X.columns
        self.cat_cols = [col for col in X.columns if X[col].dtype == 'object']
    
        self.OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        trans_data = self.OH_encoder.fit_transform(X[cat_cols])

        encode_col = self.OH_encoder.get_feature_names_out(input_features=self.cat_cols)
        self.passthrough_col = [col for col in X.columns if col not in self.cat_cols]
        self.new_cols = list(encode_col) + self.passthrough_col
        
        data = np.hstack((trans_data, X[self.passthrough_col].values))

        X_enc = pd.DataFrame(data,columns = self.new_cols)
    
        sm = SMOTEENN(random_state = 42) 
        X_tgt, y_tgt = sm.fit_resample(X_enc, y)
    
        self.cols = X_enc.columns # keeping the names of all columns

        self.scaler = MinMaxScaler()
        Xtgt_scaled = self.scaler.fit_transform(X_tgt)
        Xtgt_scaled = pd.DataFrame(Xtgt_scaled, columns=self.cols)
    
        self.X_train ,self.X_test, self.y_train, self.y_test = train_test_split(Xtgt_scaled, y_tgt, test_size=0.2)
    
        model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=0.5, gamma=0.2, gpu_id=-1,
                  importance_type='gain', interaction_constraints='',
                  learning_rate=0.1, max_delta_step=0, max_depth=8,
                  min_child_weight=1, missing=np.nan, monotone_constraints='()',
                  n_estimators=165, n_jobs=4, num_parallel_tree=1, random_state=0,
                  reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
                  tree_method='exact', use_label_encoder=False,
                  validate_parameters=1, verbosity = 0, eval_metric='logloss')
        
    
        self.model_train = model.fit(self.X_train, self.y_train)
    
    def predict(self, val=None):
        
        if val is None:
            result = self.model_train.predict(self.X_test)
            report = classification_report(self.y_test, result)
            return print(report)
        
        elif isinstance(val, np.ndarray):
            val = pd.DataFrame(val, columns=self.col)
            val_enc = pd.DataFrame(self.OH_encoder.transform(val[self.cat_cols]), columns=self.OH_encoder.get_feature_names_out(input_features=self.cat_cols))
            val_enc[self.passthrough_col] = val[self.passthrough_col]
            
            val_scaled = self.scaler.transform(val_enc)
            print(val_scaled)
            val_scaled = pd.DataFrame(val_scaled, columns=self.cols)
            
            result = self.model_train.predict(val_scaled)
            return result
        
        elif isinstance(val, pd.DataFrame):
            val_enc = pd.DataFrame(self.OH_encoder.transform(val[self.cat_cols]), columns=self.OH_encoder.get_feature_names_out(input_features=self.cat_cols))
            val_enc[self.passthrough_col] = val[self.passthrough_col]
            
            val_scaled = self.scaler.transform(val_enc)
            print(val_scaled)
            val_scaled = pd.DataFrame(val_scaled, columns=self.cols)
            
            result = self.model_train.predict(val_scaled)
            
            return result 
    
    

In [26]:
pipe = CustomPipeline(X,y.iloc[:,0])
pipe2 = CustomPipeline(X, y.iloc[:,1])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [27]:
pred = pipe.predict(np.array([['Male', 50, 'No', 'No', 4, 'Yes', 'Yes', 'No', 'Yes',
       'Month-to-Month', 'Yes', 'Bank Withdrawal', 73.9, 415.45, 'No',
       'No', 'No', 'Yes', 'No', 'No', 'No']]))


[[0.         1.         1.         0.         1.         0.
  0.         1.         0.         1.         1.         0.
  0.         1.         1.         0.         0.         0.
  1.         1.         0.         0.         1.         0.
  1.         0.         1.         0.         0.         1.
  1.         0.         1.         0.         1.         0.
  0.50819672 0.04225352 0.65165049 0.03289477]]


In [28]:
pred

array([1])

In [29]:
pred2 = pipe2.predict(np.array([['Male', 50, 'No', 'No', 4, 'Yes', 'Yes', 'No', 'Yes',
       'Month-to-Month', 'Yes', 'Bank Withdrawal', 73.9, 415.45, 'No',
       'No', 'No', 'Yes', 'No', 'No', 'No']]))

[[0.         1.         1.         0.         1.         0.
  0.         1.         0.         1.         1.         0.
  0.         1.         1.         0.         0.         0.
  1.         1.         0.         0.         1.         0.
  1.         0.         1.         0.         0.         1.
  1.         0.         1.         0.         1.         0.
  0.50819672 0.04225352 0.65165049 0.03293602]]


In [30]:
pred2[0]

2

In [31]:
category_mapping = {'Competitor': 1, 'Dissatisfaction': 2, 'Attitude': 3, 'Price': 4, 'Other': 5}

if pred[0]==1:
    val = pred2[0]
    for key, valu in category_mapping.items():
        if valu==val:
            print(f'{key}')
    

Dissatisfaction


In [32]:
model_df.iloc[2].values

array(['0004-TLHLJ', 'Male', 50, 'No', 'No', 4, 'Yes', 'Yes', 'No', 'Yes',
       'Month-to-Month', 'Yes', 'Bank Withdrawal', 73.9, 415.45, 'No',
       'No', 'No', 'Yes', 'No', 'No', 'No', 1, 1], dtype=object)

In [33]:
model_df.head(5)

,Customer ID,Gender,Age,Married,Number of Dependents,Tenure in Months,Phone Service,Internet Service,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Revenue,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Customer Status,Churn Category
0,0002-ORFBO,Female,37,Yes,No,9,Yes,Yes,No,Yes,One Year,Yes,Credit Card,65.6,974.81,No,No,Yes,No,Yes,Yes,No,0,0
1,0003-MKNFE,Male,46,No,No,9,Yes,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.0,610.28,Yes,No,No,No,No,No,Yes,0,0
2,0004-TLHLJ,Male,50,No,No,4,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.9,415.45,No,No,No,Yes,No,No,No,1,1
3,0011-IGKFF,Male,78,Yes,No,13,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1599.51,No,No,Yes,Yes,No,Yes,Yes,1,2
4,0013-EXCHZ,Female,75,Yes,No,3,Yes,Yes,No,Yes,Month-to-Month,Yes,Credit Card,83.9,289.54,No,No,No,No,Yes,Yes,No,1,2


In [34]:
preds = pipe.predict()

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       504
           1       0.97      0.96      0.97       707

    accuracy                           0.96      1211
   macro avg       0.96      0.96      0.96      1211
weighted avg       0.96      0.96      0.96      1211



In [35]:
 data = {
            'Gender': ['Male'],'Age': [50],'Married': ['No'],'Number of Dependents': ['No'],'Tenure in Months': [4],
            'Phone Service': ['Yes'],'Internet Service': ['Yes'],'Streaming Music': ['No'],'Unlimited Data': ['Yes'],
            'Contract': ['Month-to-Month'],'Paperless Billing': ['Yes'],'Payment Method': ['Bank Withdrawal'],'Monthly Charge': [73],
            'Total Revenue': [415.55],'MultipleLines': ['No'],'TechSupport': ['No'],'OnlineBackup': ['No'],
            'DeviceProtection': ['Yes'],'OnlineSecurity': ['No'],'StreamingTV': ['No'],'StreamingMovies': ['No']}

predss_data = pd.DataFrame(data)

In [36]:
st.markdown("""<div style='background-color: #c1d6c1; padding: 100px; border-radius: 5px'>
                            <h2 style='text-align: center'>{}</h2>
                            <p style='font-size: 24px; text-align: center'>Prediction</p></div>""".format('Churn'),
                                unsafe_allow_html=True)

NameError: name 'st' is not defined

In [37]:
model_df

,Customer ID,Gender,Age,Married,Number of Dependents,Tenure in Months,Phone Service,Internet Service,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Revenue,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Customer Status,Churn Category
0,0002-ORFBO,Female,37,Yes,No,9,Yes,Yes,No,Yes,One Year,Yes,Credit Card,65.60,974.81,No,No,Yes,No,Yes,Yes,No,0,0
1,0003-MKNFE,Male,46,No,No,9,Yes,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.00,610.28,Yes,No,No,No,No,No,Yes,0,0
2,0004-TLHLJ,Male,50,No,No,4,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.90,415.45,No,No,No,Yes,No,No,No,1,1
3,0011-IGKFF,Male,78,Yes,No,13,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.00,1599.51,No,No,Yes,Yes,No,Yes,Yes,1,2
4,0013-EXCHZ,Female,75,Yes,No,3,Yes,Yes,No,Yes,Month-to-Month,Yes,Credit Card,83.90,289.54,No,No,No,No,Yes,Yes,No,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Female,20,No,No,13,Yes,Yes,Yes,Yes,One Year,No,Credit Card,55.15,1349.74,No,Yes,No,No,Yes,No,No,0,0
7039,9992-RRAMN,Male,40,Yes,No,22,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,85.10,2230.10,Yes,No,No,No,No,No,Yes,1,2
7040,9992-UJOEL,Male,22,No,No,2,Yes,Yes,No,Yes,Month-to-Month,Yes,Credit Card,50.30,129.99,No,No,Yes,No,No,No,No,0,0
7041,9993-LHIEB,Male,21,Yes,No,67,Yes,Yes,Yes,Yes,Two Year,No,Credit Card,67.85,4769.69,No,Yes,No,Yes,Yes,No,Yes,0,0


In [38]:
proba = model_target1.predict_proba(scaler.fit_transform(X_enc))

In [39]:
model_target1.predict(scaler.fit_transform(X_enc))[:100]

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1])

In [40]:
model_df.head(10)

,Customer ID,Gender,Age,Married,Number of Dependents,Tenure in Months,Phone Service,Internet Service,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Revenue,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Customer Status,Churn Category
0,0002-ORFBO,Female,37,Yes,No,9,Yes,Yes,No,Yes,One Year,Yes,Credit Card,65.60,974.81,No,No,Yes,No,Yes,Yes,No,0,0
1,0003-MKNFE,Male,46,No,No,9,Yes,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.00,610.28,Yes,No,No,No,No,No,Yes,0,0
2,0004-TLHLJ,Male,50,No,No,4,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.90,415.45,No,No,No,Yes,No,No,No,1,1
3,0011-IGKFF,Male,78,Yes,No,13,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.00,1599.51,No,No,Yes,Yes,No,Yes,Yes,1,2
4,0013-EXCHZ,Female,75,Yes,No,3,Yes,Yes,No,Yes,Month-to-Month,Yes,Credit Card,83.90,289.54,No,No,No,No,Yes,Yes,No,1,2
5,0013-MHZWF,Female,23,No,Yes,9,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Credit Card,69.40,722.38,No,No,No,No,Yes,Yes,Yes,0,0
6,0013-SMEOE,Female,67,Yes,No,71,Yes,Yes,Yes,Yes,Two Year,Yes,Bank Withdrawal,109.70,8611.41,No,Yes,Yes,Yes,Yes,Yes,Yes,0,0
7,0014-BMAQU,Male,52,Yes,No,63,Yes,Yes,No,No,Two Year,Yes,Credit Card,84.65,6214.28,Yes,Yes,No,No,Yes,No,No,0,0
8,0015-UOCOJ,Female,68,No,No,7,Yes,Yes,No,Yes,Two Year,Yes,Bank Withdrawal,48.20,414.06,No,Yes,No,No,No,No,No,0,0
9,0016-QLJIS,Female,43,Yes,Yes,65,Yes,Yes,Yes,Yes,Two Year,Yes,Credit Card,90.45,7807.80,Yes,Yes,Yes,Yes,Yes,Yes,Yes,0,0


In [41]:
proba = np.round(proba, 2)

In [42]:
probs = proba[:,1]

In [43]:
model_df['Probs'] = probs

/var/folders/1v/57gd2jmd04lg8mxrdxd44x300000gn/T/ipykernel_48258/3624725332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Probs'] = probs


In [44]:
proba.shape

(7043, 2)

In [45]:
model_df['Probs'] = model_df['Probs'].multiply(100)

/var/folders/1v/57gd2jmd04lg8mxrdxd44x300000gn/T/ipykernel_48258/1041605927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Probs'] = model_df['Probs'].multiply(100)


In [46]:
model_df.to_csv('prob_df')

In [47]:
model_df[model_df['Customer ID']=='0013-MHZWF']['Age'].values[0]

23

In [48]:
df.drop(columns=['Unnamed: 0','Number of Dependents','Zip Code','Latitude','Longitude','Number of Referrals','Offer','Phone Service'],inplace=True)

KeyError: "['Unnamed: 0', 'Zip Code', 'Latitude', 'Longitude', 'Offer'] not found in axis"

In [49]:
df.drop(columns=['Internet Service','Streaming Music','Paperless Billing','Payment Method','Total Refunds','Total Extra Data Charges'],inplace=True)

In [50]:
df.drop(columns=['Internet Type','Churn Category','MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies'],inplace=True)

In [51]:
df['Probs'] = probs

In [52]:
df['Probs']=df['Probs'].multiply(100)

In [53]:
df.to_csv('prob_df')

In [54]:
df[df['Probs']>60]['Churn Reason']

KeyError: 'Churn Reason'

In [ ]:
'ANIRBAN SARKAR'

In [601]:
df.columns

Index(['Customer ID', 'Gender', 'Age', 'Married', 'City', 'Tenure in Months',
       'Avg Monthly Long Distance Charges', 'Avg Monthly GB Download',
       'Unlimited Data', 'Contract', 'Monthly Charge', 'Total Charges',
       'Total Long Distance Charges', 'Total Revenue', 'Customer Status',
       'Churn Reason', 'Probs'],
      dtype='object')

In [617]:
cust_id = '0011-IGKFF'

In [618]:
filtered_df = df[(df['Customer ID'] == cust_id) & (df['Probs'] > 65) & (~df['Churn Reason'].isna())]

In [619]:
filtered_df

,Customer ID,Gender,Age,Married,City,Tenure in Months,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Unlimited Data,Contract,Monthly Charge,Total Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Reason,Probs
3,0011-IGKFF,Male,78,Yes,Martinez,13,27.82,4.0,Yes,Month-to-Month,98.0,1237.85,361.66,1599.51,Churned,Product dissatisfaction,100.0


In [623]:
df['Churn Reason'].value_counts().index

Index(['Competitor had better devices', 'Competitor made better offer',
       'Attitude of support person', 'Don't know',
       'Competitor offered more data',
       'Competitor offered higher download speeds',
       'Attitude of service provider', 'Price too high',
       'Product dissatisfaction', 'Network reliability',
       'Long distance charges', 'Service dissatisfaction', 'Moved',
       'Extra data charges', 'Limited range of services',
       'Poor expertise of online support',
       'Lack of affordable download/upload speed',
       'Lack of self-service on Website', 'Poor expertise of phone support',
       'Deceased'],
      dtype='object')

In [56]:
prob_df = pd.read_csv('/Users/ruruthapa/Desktop/ML/machinelearning/Telecom_Churn/venv/repository/prob_df')

In [58]:
prob_df['Avg Monthly GB Download']

0       16.0
1       10.0
2       30.0
3        4.0
4       11.0
        ... 
7038    59.0
7039    17.0
7040    51.0
7041    58.0
7042     5.0
Name: Avg Monthly GB Download, Length: 7043, dtype: float64